### Importações

In [1]:
import requests
from google.colab import userdata
import psycopg2
from psycopg2 import extras
import pandas as pd

### Autenticando API e testando endpoint

In [ ]:
token = userdata.get('token_br')

teste = requests.get("https://api.brasil.io/v1/dataset/cursos-prouni/cursos/data/", headers={"Authorization": f"Token {token}", "User-Agent": "python-requests/brasilio-client-0.1.0"
})

if teste.status_code == 200:
    print("Conexão bem-sucedida!")
    print(teste.json())
else:
    print(f"Erro: {teste.status_code}")
    print(teste.text)


Conexão bem-sucedida!
{'count': 41447, 'next': 'https://api.brasil.io/v1/dataset/cursos-prouni/cursos/data/?page=2', 'previous': None, 'results': [{'bolsa_integral_ampla': 1, 'bolsa_integral_cotas': 1, 'bolsa_parcial_ampla': None, 'bolsa_parcial_cotas': None, 'campus_external_id': 9912, 'campus_nome': 'ACRELANDIA - Centro', 'cidade_busca': 'Acrelandia', 'grau': 'Bacharelado', 'mensalidade': '289.00', 'nome': 'Administração', 'nota_integral_ampla': '572.74', 'nota_integral_cotas': '548.00', 'nota_parcial_ampla': None, 'nota_parcial_cotas': None, 'turno': 'Curso a Distância', 'uf_busca': 'AC', 'universidade_nome': 'Universidade Paulista - UNIP'}, {'bolsa_integral_ampla': None, 'bolsa_integral_cotas': 1, 'bolsa_parcial_ampla': None, 'bolsa_parcial_cotas': None, 'campus_external_id': 6235, 'campus_nome': 'PAP CRUZEIRO DO SUL - Centro', 'cidade_busca': 'Cruzeiro do Sul', 'grau': 'Bacharelado', 'mensalidade': '298.00', 'nome': 'Administração', 'nota_integral_ampla': '646.14', 'nota_integral_

### Dados do database

In [5]:
database_config = {
    'dbname': 'grupo_1',
    'user': 'grupo_1',
    'password': 'how_ebanx',
    'host': 'how.c3gus406qdox.eu-north-1.rds.amazonaws.com',
    'port': '5432'
}

### Conectando com o banco de dados e executando uma query DDL para criação da tabela

In [ ]:
with psycopg2.connect(**database_config) as conn:
    with conn.cursor() as cur:
        query = """
          CREATE TABLE IF NOT EXISTS curso_prouni (
            UF VARCHAR,
            Cidade VARCHAR,
            Universidade VARCHAR,
            Curso VARCHAR,
            TURNO VARCHAR,
            MENSALIDADE FLOAT,
            BOLSA_INTEGRAL_AMPLA FLOAT,
            NOTA_INTEGRAL_AMPLA FLOAT,
            BOLSA_INTEGRAL_COTA FLOAT,
            NOTA_INTEGRAL_COTA FLOAT
          );
        """
        cur.execute(query)
        conn.commit()


### Funções que buscam os dados no endpoint e adicionam através de bulk insert por página

In [ ]:

url_base = "https://api.brasil.io/v1/dataset/cursos-prouni/cursos/data/" #URL base utilizada para extrair os dados

def processar_paginas(): #Função principal - percorre as páginas e chama a função dados_pagina, se houver dados, chama a função insert_sql
    paginas = 42
    for page in range(1, paginas + 1):
        print(f"Processando página {page} de {paginas}...")
        dados = dados_pagina(page)
        if dados:
            insert_sql(dados)
        else:
            print(f"Nenhum dado encontrado na página {page}.")

def dados_pagina(page): # Função responsável por acessar os dados do endpoint por página e retornar os valores para a função processar_paginas
    url = f"{url_base}?page={page}"
    response = requests.get(url, headers= {"Authorization": f"Token {token}", "User-Agent": "python-requests/brasilio-client-0.1.0"})
    if response.status_code == 200:
        return response.json()["results"]
    else:
        print(f"Erro ao buscar página {page}: {response.status_code}")
        return []

def insert_sql(data): # Função responsável por inserir os dados recebidos pela função dados_pagina no banco de dados através de bulk insert
    with psycopg2.connect(**database_config) as conn:
        with conn.cursor() as cur:
            query = """
            INSERT INTO curso_prouni (UF, Cidade, Universidade, Curso, TURNO, MENSALIDADE, BOLSA_INTEGRAL_AMPLA, NOTA_INTEGRAL_AMPLA, BOLSA_INTEGRAL_COTA, NOTA_INTEGRAL_COTA)
            VALUES %s
            """
            records = [
                (
                    item["uf_busca"],
                    item["cidade_busca"],
                    item["universidade_nome"],
                    item["nome"],
                    item["turno"],
                    item["mensalidade"],
                    item["bolsa_integral_ampla"],
                    item["nota_integral_ampla"],
                    item["bolsa_integral_cotas"],
                    item["nota_integral_cotas"],
                )
                for item in data
            ]
            extras.execute_values(cur, query, records)
            conn.commit()




In [ ]:
if __name__ == "__main__": #Função para executar o processo acima
    processar_paginas()

### Queries estratégicas

In [13]:
query_1 = """
SELECT curso, uf, universidade,
       nota_integral_ampla,
       nota_integral_cota,
       bolsa_integral_ampla,
       bolsa_integral_cota
FROM public.curso_prouni
WHERE curso = 'Medicina'
ORDER BY nota_integral_ampla;
"""

query_2 = """
SELECT curso, uf,
       AVG(nota_integral_ampla) AS media_integral_ampla,
       AVG(nota_integral_cota) AS media_integral_cota
FROM public.curso_prouni
WHERE curso = 'Medicina'
GROUP BY curso, uf; """

query_3 = """
SELECT curso, ROUND(AVG(mensalidade)::numeric, 2) AS media_mensalidade ,uf, COUNT(*)
FROM public.curso_prouni
WHERE curso = 'Medicina'
GROUP BY curso,uf
HAVING AVG(mensalidade) IS NOT NULL
ORDER BY media_mensalidade DESC;
"""

### Função de consulta que retorna o resultado em tabela de uma das 3 queries acima

In [14]:
def consulta(query):
    with psycopg2.connect(**database_config) as conn:
        with conn.cursor() as cur:
            return pd.read_sql_query(query, conn)

In [16]:
consulta(query_1)

<ipython-input-14-f1dea2e35d14>:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


,curso,uf,universidade,nota_integral_ampla,nota_integral_cota,bolsa_integral_ampla,bolsa_integral_cota
0,Medicina,BA,Faculdade Ages de Medicina - Faculdade AGES,732.70,NaN,NaN,2.0
1,Medicina,RS,Universidade Regional Integrada do Alto Urugua...,733.40,717.70,4.0,1.0
2,Medicina,ES,Universidade VIla Velha - UVV,734.26,NaN,NaN,1.0
3,Medicina,SP,Claretiano - Faculdade - Claretianorc - CLARET...,734.44,720.86,9.0,2.0
4,Medicina,SP,Centro Universitário Lusíada - UNILUS,734.46,723.84,10.0,6.0
...,...,...,...,...,...,...,...
119,Medicina,PA,Centro Universitário do Estado do Pará - Cesup...,772.40,736.00,1.0,3.0
120,Medicina,PI,"Faculdade de Ciências Humanas,Exatas e da Saúd...",772.82,739.86,1.0,4.0
121,Medicina,PR,Faculdade Campo Real - CAMPO REAL,781.20,731.04,1.0,1.0
122,Medicina,ES,Centro Universitário do Espírito Santo - Unesc...,NaN,NaN,NaN,NaN
